# Creación de su primera campaña

Este cuaderno lo guiará por los pasos para crear un modelo de recomendación para películas basado en los datos recolectados desde el conjunto de datos de MovieLens. El objetivo es recomendar películas relevantes para un usuario concreto.

Los datos provienen del proyecto MovieLens. Puede aprender más sobre los datos y los posibles usos mediante una búsqueda en la web durante cualquiera de los periodos de espera en las celdas que aparecen abajo.

## Cómo utilizar el cuaderno

El código se divide en celdas como la siguiente. En la parte superior de esta página, se encuentra un botón triangular `Run` (Ejecutar) en el que puede hacer clic para ejecutar cada celda y pasar a la siguiente, o puede presionar `Shift` + `Enter`mientras está en la celda para ejecutarla y pasar a la siguiente.

A medida que una celda se ejecuta, verá una línea al costado que muestra un `*` mientras la celda se está ejecutando o se actualizará a un número para indicar la última celda que terminó de ejecutarse luego de que haya terminado de ejecutar todo el código dentro de una celda.


Solo debe seguir las instrucciones que aparecen a continuación y ejecutar las celdas para comenzar a utilizar Amazon Personalize.

## Importaciones 

Python viene con una amplia colección de bibliotecas y necesitamos importarlas, así como las que se instalan para ayudarnos, como [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK para Python) y [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/), que son herramientas básicas de la ciencia de datos.

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
!conda install -y -c conda-forge unzip

A continuación, querrá validar que su entorno puede comunicarse correctamente con Amazon Personalize. Las líneas que figuran a continuación hacen precisamente eso.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## Configurar los datos

Los datos se importan a Amazon Personalize a través de Amazon S3. A continuación, especificaremos un bucket creado dentro de AWS para este ejercicio.

A continuación, se actualizará la variable `bucket` para que en su lugar se establezca el valor que creó antes en los pasos de CloudFormation. Esto debe estar en un archivo de texto de su trabajo anterior. No es necesario cambiar el `filename`.

### Especificar un bucket y una ubicación de salida de datos
Asegúrese de actualizar el valor `bucket` si lo personalizó en la etapa de implementación de la plantilla de CloudFormation. Haga clic en la celda que aparece a continuación para realizar cambios.

In [ ]:
bucket = "personalizedemofirstnamelastname"       # replace with the name of your S3 bucket
filename = "movie-lens-100k.csv"

### Descargar, preparar y cargar los datos de entrenamiento

En la actualidad, aún no tiene los datos de MovieLens cargados de forma local para examinarlos. Ejecute las siguientes líneas para descargar la última copia y examinarla rápidamente.

#### Descargar y explorar el conjunto de datos

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

#### Preparar y cargar los datos

Como puede ver, los datos contienen un UserID, un ItemID, una clasificación y una marca temporal.

Ahora eliminaremos los elementos con clasificaciones bajas y también la columna Rating (Clasificación) antes de crear nuestro modelo.

Una vez listo, guardaremos el archivo como un CSV nuevo y lo cargaremos en S3.

Ejecute las líneas en la celda que aparece a continuación para completar este paso.

In [ ]:
data = data[data['RATING'] > 3]                # Keep only movies rated higher than 3 out of 5.
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

### Crear un esquema

Un componente central de cómo Personalize entiende sus datos proviene del esquema que se define a continuación. Esta configuración indica al servicio cómo digerir los datos proporcionados a través de su archivo CSV. Observe que las columnas y los tipos se alinean con lo que había en el archivo que creó anteriormente.

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-demo-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

### Crear y esperar el grupo de conjuntos de datos

La agrupación más grande en Personalize es un grupo de conjuntos de datos, esto aislará sus datos, rastreadores de eventos, soluciones y campañas. Agrupe elementos que comparten una colección de datos común. Siéntase libre de modificar el nombre que aparece a continuación si lo desea.

#### Crear un grupo de conjuntos de datos

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-launch-demo"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### Esperar a que el grupo de conjunto de datos tenga estado ACTIVO

Debe estar activo antes de que podamos utilizar el grupo de conjunto de datos en cualquiera de los siguientes artículos. Ejecute la celda a continuación y espere a que se muestre activo.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### Crear conjuntos de datos

Después del grupo, el próximo paso es crear los conjuntos de datos reales. En este ejemplo, crearemos 1 para los datos de las interacciones. Ejecute las celdas a continuación para crearlo.

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-launch-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

#### Adjuntar la política al bucket de S3

Amazon Personalize debe ser capaz de leer el contenido del bucket de S3 que creó antes. Las líneas que aparecen a continuación se encargarán de eso.

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

#### Crear un rol de Personalize

Además, Amazon Personalize necesita la capacidad de asumir roles en AWS de modo que tenga los permisos para ejecutar ciertas tareas. Las líneas que aparecen a continuación otorgan eso.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleDemo"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## Importar los datos

Anteriormente, creó el grupo de conjunto de datos y el conjunto de datos para alojar su información. Ahora ejecutará un trabajo de importación que cargará los datos de S3 en Amazon Personalize para su uso, a medida que crea su modelo.

#### Crear un trabajo de importación de conjuntos de datos

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-demo-import1",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

#### Esperar a que el trabajo de importación de conjuntos de datos tenga un estado ACTIVO

Puede pasar un tiempo antes de que el trabajo de importación se complete. Espere hasta que vea que está activo a continuación.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Crear la solución y la versión

En Amazon Personalize, un modelo entrenado se denomina solución, cada solución puede tener muchas versiones específicas relacionadas con un volumen determinado de datos cuando se entrenó el modelo.

Para comenzar, enumeraremos todas las recetas que se admiten. Una receta es un algoritmo que aún no ha sido entrenado en sus datos. Después del listado, seleccionará uno y lo utilizará para construir su modelo.

### Seleccionar receta

In [ ]:
list_recipes_response = personalize.list_recipes()
list_recipes_response

#### User Personalization
La receta [User-Personalization](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html) (aws-user-personalization) está optimizada para todos los escenarios de recomendación USER_PERSONALIZATION. Utiliza la exploración automática de elementos para hacer recomendaciones.

Con la exploración automática, Amazon Personalize prueba automáticamente diferentes recomendaciones de elementos, aprende de la forma en que los usuarios interactúan con estos elementos recomendados y potencia las recomendaciones de elementos que impulsan un mejor compromiso y conversión. Esto mejora el descubrimiento de elementos y la participación cuando tiene un catálogo que cambia rápidamente, o cuando los nuevos elementos, como las noticias o las promociones, son más relevantes para los usuarios cuando están frescos.

Puede equilibrar cuánto explorar (donde los elementos con menos datos de interacciones o relevancia se recomiendan con más frecuencia) frente a cuánto explotar (donde las recomendaciones se basan en lo que conocemos o en la relevancia). Amazon Personalize ajusta automáticamente las futuras recomendaciones en función de los comentarios implícitos de los usuarios.

En primer lugar, seleccione la receta buscando el ARN en la lista de recetas anterior.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization" # aws-user-personalization selected for demo purposes

### Crear y esperar una solución

Primero creará la solución con la API, luego creará una versión. Tardará varios minutos en entrenar el modelo y crear así su versión de una solución. Una vez que se pone en marcha y se ven las notificaciones en curso es un buen momento para tomar un descanso, tomar un café, etc.

#### Crear una solución

In [ ]:
create_solution_response = personalize.create_solution(
    name = "personalize-demo-soln-user-personalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### Crear una versión de la solución

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

#### Esperar a que la versión de la solución esté ACTIVA

Esto tardará entre 40 y 50 minutos aproximadamente.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### Obtener las métricas de la versión de la solución

Una vez que disponga de su solución y de su versión, puede obtener las métricas para juzgar su rendimiento. Estas métricas no son especialmente buenas, ya que se trata de un conjunto de datos de demostración, pero con conjuntos de datos más grandes y complejos se deberían ver mejoras.

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

Recomendamos leer [la documentación](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) para comprender las métricas, pero, además, copiamos partes de la documentación a continuación para mayor comodidad.

Es necesario que comprenda los siguientes términos relativos a la evaluación en Personalize:

- *Recomendación relevante* se refiere a una recomendación que coincide con un valor en los datos de prueba para el usuario en particular.
- *Rango* se refiere a la posición de un elemento recomendado en la lista de recomendaciones. Se supone que la posición 1 (la primera de la lista) es la más relevante para el usuario.
- *Consulta* se refiere al equivalente interno de una llamada a GetRecommendations.

Las métricas producidas por Personalize son las siguientes:

- coverage: La proporción de elementos únicos recomendados de todas las consultas sobre el número total de elementos únicos en los datos de entrenamiento (incluye tanto los conjuntos de datos de elementos como de interacciones).
- mean_reciprocal_rank_at_25: La [media de los rangos recíprocos](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) de la primera recomendación relevante entre las 25 primeras recomendaciones sobre todas las consultas. Esta métrica es apropiada si está interesado en la recomendación de mayor rango.
- normalized_discounted_cumulative_gain_at_K: La ganancia descontada supone que las recomendaciones más bajas de una lista de recomendaciones son menos relevantes que las más altas. Por lo tanto, cada recomendación se descuenta (se le da un peso menor) por un factor que depende de su posición. A fin de obtener la [ganancia descontada acumulada](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) en K, se suma cada recomendación descontada relevante en las  recomendaciones más importantes de K. La ganancia acumulada descontada normalizada (NDCG) es la DCG dividida por la DCG ideal, de manera que la NDCG está entre 0 y 1. (El DCG ideal es aquel en el que las recomendaciones más importantes de K están ordenadas por relevancia). Amazon Personalize utiliza un factor de ponderación de 1/log (1 + posición), donde la parte superior de la lista es la posición 1. Esta métrica recompensa los elementos relevantes que aparecen cerca de la parte superior de la lista, porque la parte superior de una lista suele llamar más la atención.
- precision_at_K: El número de recomendaciones relevantes de las principales recomendaciones de K dividido por K. Esta métrica recompensa la recomendación precisa de los elementos relevantes.

## Crear y esperar una campaña

Ahora que tiene una versión de la solución que funciona, tendrá que crear una campaña para utilizarla con sus aplicaciones. Una campaña es una versión de la solución alojada; un punto de conexión que puede consultar para obtener recomendaciones. El precio se fija estimando la capacidad de rendimiento (solicitudes de personalización de los usuarios por segundo). Cuando implementa una campaña, se establece un valor mínimo de transacciones por segundo (TPS) (`minProvisionedTPS`). Este servicio, como muchos dentro de AWS, se escalará automáticamente en función de la demanda, pero si la latencia es crítica, es posible que desee aprovisionar por adelantado para una mayor demanda. Para esta demostración, todos los límites mínimos de rendimiento se establecen en 1. Para obtener más información, consulte la página de [precios](https://aws.amazon.com/personalize/pricing/).

Como se ha mencionado anteriormente, la receta de personalización del usuario utilizada para nuestra solución admite la exploración automática de elementos "fríos". Puede controlar el grado de exploración que se realiza cuando crea su campaña. El tipo de datos `itemExplorationConfig` admite parámetros `explorationWeight` y `explorationItemAgeCutOff`. El peso de la exploración determina la frecuencia con la que las recomendaciones incluyen elementos con menos datos de interacción o relevancia. Cuanto más se acerque el valor a 1,0, mayor será la exploración. A cero, no se produce ninguna exploración y las recomendaciones se basan en los datos actuales (relevancia). El límite de antigüedad de los elementos de exploración determina los elementos que deben explorarse en función del tiempo transcurrido desde la última interacción. Proporcione la antigüedad máxima del elemento, en días desde la última interacción, para definir el alcance de la exploración del elemento. Cuanto mayor sea el valor, más elementos se tendrán en cuenta durante la exploración. Para nuestra campaña, especificaremos un peso de exploración de 0,5.

#### Crear una campaña

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-demo-camp",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1,
    campaignConfig = {
        "itemExplorationConfig": {
            "explorationWeight": "0.5"
        }
    }
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

#### Esperar a que la campaña tenga un estado ACTIVO

Esto debería llevar unos 10 minutos.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

## Obtener muestras de recomendaciones

Cuando la campaña esté activa, estará lista para recibir recomendaciones. Primero tenemos que seleccionar un usuario de la colección al azar. A continuación, crearemos algunas funciones de ayuda para obtener la información de la película que se mostrará para las recomendaciones en vez de mostrar únicamente los ID.

In [ ]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

In [ ]:
# First load items into memory
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.iloc[movie_id]['TITLE']


#### Llamar a GetRecommendations

Con el usuario que obtuvo anteriormente, las líneas siguientes le proporcionarán recomendaciones y le mostrarán la lista de películas recomendadas.


In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['OriginalRecs'])
recommendations_df

## Revisión

Con los códigos anteriores, logró entrenar correctamente un modelo de aprendizaje profundo para generar recomendaciones de elementos basadas en el comportamiento previo del usuario. Piense en otros tipos de problemas donde tiene disponible estos datos y cómo sería crear un sistema como este para ofrecer esas recomendaciones.

Ya está listo para avanzar al siguiente cuaderno `2.View_Campaign_And_Interactions.ipynb`



## Notas para el próximo cuaderno:

Hay algunos valores que necesitará para el próximo cuaderno. Ejecute las celdas que aparecen a continuación para almacenarlos y así poder copiarlos y pegarlos en la siguiente parte del ejercicio.

In [ ]:
%store campaign_arn

In [ ]:
%store dataset_group_arn

In [ ]:
%store solution_version_arn

In [ ]:
%store solution_arn

In [ ]:
%store dataset_arn

In [ ]:
%store campaign_arn

In [ ]:
%store schema_arn

In [ ]:
%store bucket

In [ ]:
%store filename

In [ ]:
%store role_name

In [ ]:
%store recommendations_df

In [ ]:
%store user_id